# 沒寫完predict 與 output.csv

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_x=pd.read_csv('X_train').values
train_y=pd.read_csv('Y_train').values
test_x=pd.read_csv('X_test').values

# data perprocess

In [3]:
# normalize
mean=np.mean(train_x,axis=0).reshape(1,-1)
std=np.std(train_x,axis=0).reshape(1,-1)

train_x=((train_x-mean)/std)


In [4]:
train_y.shape

(32561, 1)

In [5]:
class_0_id=list()
class_1_id=list()
for i in range(len(train_y)):
    if train_y[i][0] == 0:
        class_0_id.append(i)
    else:
        class_1_id.append(i)

In [6]:
class_0_feature=train_x[class_0_id]
class_1_feature=train_x[class_1_id]

In [7]:
print('class_0_feature shape:', class_0_feature.shape)
print('class_1_feature shape:', class_1_feature.shape)

class_0_feature shape: (24720, 106)
class_1_feature shape: (7841, 106)


# generative model

In [9]:
%%time
# class 1
mu_0=np.mean(class_0_feature,axis=0) 
mu_0=mu_0.reshape(1,mu_0.shape[0])
mu_0=mu_0.repeat(class_0_feature.shape[0],axis=0)
# print("mu_0.shape:",mu_0.shape)

n = class_0_feature.shape[1]
sigma_0 = np.zeros((n,n))
for i in range(class_0_feature.shape[0]):
    # 問題出在這個內積完以後是(106,)而不是(1,106)
    temp=class_0_feature[i]-mu_0[i]
    temp=temp.reshape(1,-1)
    sigma_0 += np.dot(temp.T, temp) / class_0_feature.shape[0]

    
print(sigma_0.shape)

# class 2
mu_1=np.mean(class_1_feature,axis=0)
mu_1=mu_1.reshape(1,mu_1.shape[0])
mu_1=mu_1.repeat(class_1_feature.shape[0],axis=0)
print("mu_1.shape:",mu_1.shape)

n = class_1_feature.shape[1]
sigma_1 = np.zeros((n,n))
for i in range(class_1_feature.shape[0]):
    temp=class_1_feature[i]-mu_1[i]
    temp=temp.reshape(1,-1)
    sigma_1 += np.dot(temp.T,temp) / class_1_feature.shape[0]


(106, 106)
mu_1.shape: (7841, 106)
CPU times: user 814 ms, sys: 3.95 ms, total: 818 ms
Wall time: 817 ms


In [45]:
from math import log,exp
sigma=(class_0_feature.shape[0]*sigma_0+class_1_feature.shape[0]*sigma_1)/(class_0_feature.shape[0]+class_1_feature.shape[0])
inv_sig=np.linalg.inv(sigma)
weight=((mu_0[0]-mu_1[0]).dot(inv_sig))
bias=(-0.5)*mu_0[0].dot(inv_sig).dot(mu_0[0].T)\
+0.5*mu_1[0].dot(inv_sig).dot(mu_1[0].T)\
+log(class_0_feature.shape[0]/class_1_feature.shape[0])

In [46]:
# 輸入維一個sample的feature
def func(x):
    global weight
    global bias
    z=weight.dot(x)+bias
    sigmoid=1/1+exp(z)